In [ ]:
# Copyright (C) 2024 Hansem Ro <hansemro@outlook.com>
# Copyright (C) 2018-2020 Claire Xenia Wolf <claire@yosyshq.com>
# Copyright (C) 2018-2020 gatecat <gatecat@ds0.me>
# Copyright (C) 2018-2020 Dan Gisselquist <dan@symbioticeda.com>
# Copyright (C) 2018-2020 Serge Bazanski <q3k@q3k.org>
# Copyright (C) 2018-2020 Miodrag Milanovic <micko@yosyshq.com>
# Copyright (C) 2018-2020 Eddie Hung <eddieh@ece.ubc.ca>
#
# Permission to use, copy, modify, and/or distribute this software for any
# purpose with or without fee is hereby granted, provided that the above
# copyright notice and this permission notice appear in all copies.
#
# THE SOFTWARE IS PROVIDED "AS IS" AND THE AUTHOR DISCLAIMS ALL WARRANTIES
# WITH REGARD TO THIS SOFTWARE INCLUDING ALL IMPLIED WARRANTIES OF
# MERCHANTABILITY AND FITNESS. IN NO EVENT SHALL THE AUTHOR BE LIABLE FOR
# ANY SPECIAL, DIRECT, INDIRECT, OR CONSEQUENTIAL DAMAGES OR ANY DAMAGES
# WHATSOEVER RESULTING FROM LOSS OF USE, DATA OR PROFITS, WHETHER IN AN
# ACTION OF CONTRACT, NEGLIGENCE OR OTHER TORTIOUS ACTION, ARISING OUT OF
# OR IN CONNECTION WITH THE USE OR PERFORMANCE OF THIS SOFTWARE.

# nextpnr-xilinx Post-Route JSON Netlist to Vivado Design Checkpoint Conversion

Ported by: Hansem Ro

Original Java source: https://github.com/gatecat/nextpnr-xilinx/blob/xilinx-upstream/xilinx/java/json2dcp.java

Tasks:

- [x] Port nextpnr json netlist parser
- [ ] Port json2dcp
    - [x] Parse nextpnr-xilinx json netlist
        - nextpnr-himbaechel is not yet supported due to changes in NEXTPNR_BEL format
    - [x] Process each NextpnrCells
        - [x] create and place IOB cells
            - ignore PAD type cells as they are not valid RapidWright Cells.
            - use Design.createAndPlaceIOB(...) to create each IBUF/OBUF cell
                - IBUF/OBUF cells will not appear in the Vivado Device pane unless its nets are connected
        - [x] create and place other cells
        - [x] set cell properties
            - [x] fixup INIT string format
    - [x] Create and connect each NextpnrNet
        - [x] create and add net to design
        - [x] connect net to driver and user cells
    - [ ] Process each NextpnrNet PIP routing
        - [x] intra-site routing
            - use SiteInst.routeIntraSiteNet(...) to assign a net to a site wire and route it
            - use site pin name for bel name and bel pin name to create its associated BELPin
            - [x] case 1: site pin to bel pin routing
            - [x] case 2: bel pin to bel pin routing
            - [x] case 3: bel pin to site pin routing
        - [ ] inter-site routing
            - [ ] research how to manually route PIPs/nodes
                - something to do with PIP/RouteNode/Wire objects?
            - [ ] case 1: tile to site (pin) routing
            - [ ] case 2: site (pin) to tile routing
            - [ ] case 3: tile to tile routing
    - [ ] Fix whatever else is missing/broken
    - [x] write design checkpoint
- [ ] Compare NextpnrDesign and RapidWright Design netlist
- [ ] Test with various designs and architectures
    - IOB-less design: primitive-tests/clb-tests/jtag-test
    - litex

In [ ]:
import json
from argparse import ArgumentParser, FileType
import math
import re
from nextpnr import NextpnrDesign, NextpnrCell, NextpnrCellPort, NextpnrNet, PortDirection
import rapidwright
from com.xilinx.rapidwright.design import Cell, Design, Net, NetType, PinType, SitePinInst, Unisim
from com.xilinx.rapidwright.edif import EDIFDirection, EDIFHierNet, EDIFNet, EDIFPortInst, EDIFTools

from java.nio.file import Paths
from com.xilinx.rapidwright.debug import DotEdifDumper

In [ ]:
# Provide post-route json netlist produced by nextpnr-xilinx, device part, and dcp destination path
# nextpnr-himbaechel:xilinx is not supported!

json_file = "./top.route.json"
out_dcp = "./top.dcp"
part = "xc7a200tfbg484-3"

ndes = NextpnrDesign()
with open(json_file, "r") as file:
    ndes.load(json.load(file))

In [ ]:
def escape_name(s:str):
    return s.replace("\\","__").replace("/","_")

def connect_phys_and_log(net:Net, cell:Cell, logical_pin:str):
    if cell.getName().contains("/") or net.getLogicalNet() is None:
        phys_pins = cell.getAllPhysicalPinMappings(logical_pin)
        #print(f"SC1: {net}<->{logical_pin} {phys_pins}")
        for bel_pin in phys_pins:
            pin_name = cell.getBEL().getPin(bel_pin).getConnectedSitePinName()
            if pin_name:
                spi = SitePinInst(cell.getBEL().getPin(bel_pin).isOutput(), pin_name, cell.getSiteInst())
                net.addPin(spi)
                #print(f"\tSC1: connect {net.getName()} to pin {spi.getName()}")
    elif cell.getPhysicalPinMapping(logical_pin) is None or \
            cell.getBEL().getPin(cell.getPhysicalPinMapping(logical_pin)).getConnectedSitePinName() is None or \
            logical_pin.endswith("]"):
        # create logical pin only
        #print(f"SC2: {net}<->{logical_pin}")
        epi = None
        eci = cell.getEDIFCellInst()
        assert eci is not None
        if logical_pin.endswith("]"):
            open_pos = len(logical_pin) - logical_pin[::-1].index('[') - 1
            log_bus = logical_pin[0:open_pos]
            port_index = int(logical_pin[open_pos+1:len(logical_pin)-1])
            bus_width = eci.getPort(log_bus).getWidth()
            epi = net.getLogicalNet().createPortInst(log_bus, (bus_width - 1) - port_index, eci)
        else:
            epi = net.getLogicalNet().createPortInst(logical_pin, eci)
        # connect to physical pin if available
        phys_pins = cell.getAllPhysicalPinMappings(logical_pin)
        for bel_pin in phys_pins:
            pin_name = cell.getBEL().getPin(bel_pin).getConnectedSitePinName()
            if pin_name:
                spi = SitePinInst(epi.getDirection() == EDIFDirection.OUTPUT, pin_name, cell.getSiteInst())
                net.addPin(spi)
                #print(f"\tSC2: connect {net.getName()} to pin {spi.getName()}")
    else:
        #print(f"SC3: connect {net.getName()} to {logical_pin}")
        net.connect(cell, logical_pin)

def fixup_init(s:str, num_bits:int):
    hex_val = s.split("'h")[1]
    digits = max(num_bits / 4, 1)
    while (len(hex_val) < digits):
        hex_val = "0" + hex_val
    return f"{num_bits}'h{hex_val}"

# A site BEL PIP has the following format:
# SITEWIRE/<SITE>_<SITE_COORD>/<BEL>_<PIN>
def is_site_bel_pip(s:str):
    return re.match(r'^SITEWIRE/[A-Z0-9_]+_X[0-9]+Y[0-9]+/[A-Z0-9_]+_[A-Z0-9]+$', s) is not None

# A site pin PIP has the following format:
# SITEWIRE/<SITE>_<SITE_COORD>/<SITE_PIN>
def is_site_pin_pip(s:str):
    return re.match(r'^SITEWIRE/[A-Z0-9_]+_X[0-9]+Y[0-9]+/[A-Z0-9]+$', s) is not None

# Format: SITEWIRE/<SITE>_<SITE_COORD>/<BEL>_<PIN> 
def parse_site_bel_pip(s:str):
    assert is_site_bel_pip(s)
    sp = s.split("/")
    assert len(sp) == 3
    site_name = sp[1]
    bel_name,bel_pin_name = sp[2].rsplit("_",1)
    return (site_name, bel_name, bel_pin_name)

# Format: SITEWIRE/<SITE>_<SITE_COORD>/<SITE_PIN> 
def parse_site_pin_pip(s:str):
    assert is_site_pin_pip(s)
    sp = s.split("/")
    assert len(sp) == 3
    site_name = sp[1]
    site_pin_name = sp[2]
    return (site_name, site_pin_name)

# Format: <TILE>_<TILE_COORD>/<TILE_WIRE>
def parse_tile_pip(s:str):
    assert not s.startswith("SITEWIRE/")
    assert not is_site_bel_pip(s)
    assert not is_site_pin_pip(s)
    tile_name,wire_name = s.split("/")
    return (tile_name, wire_name)

In [ ]:
des = Design("top", part)

# process each NextpnrCell...
# - assigns RapidWright Cell (rwCell) to each NextpnrCell (except IOB PADs)
# - handles pin mapping between NextpnrCell's logical pin and Cell's physical pin
# - assigns cell properties/attributes
for nc in ndes.cells.values():
    uni_type = None
    bel_loc = None
    #print(f"Handling {nc.type} cell {nc.name}")
    if 'X_ORIG_TYPE' not in nc.attrs:
        continue
    if nc.type == "PAD":
        continue
    elif nc.type.startswith("IOB"):
        # Find associated PAD cell
        pad_nc = None       
        if "OUTBUF" in nc.type:
            for pad_user in nc.ports["OUT"].net.users:
                if pad_user.cell.type == "PAD":
                    pad_nc = pad_user.cell
                    break
        elif "INBUF" in nc.type:
            for pad_driver in nc.ports["PAD"].net.drivers:
                if pad_driver.cell.type == "PAD":
                    pad_nc = pad_driver.cell
                    break
        else:
            print(f"Unhandled IOB cell: {nc.type}")
            continue
        assert pad_nc is not None, f"Missing PAD cell for {nc.name}"
        pin_dir = PinType.valueOf(pad_nc.attrs["X_IO_DIR"])
        pkg_pin = pad_nc.attrs["PACKAGE_PIN"]
        io_std = pad_nc.attrs["IOSTANDARD"]
        nc.rwCell = des.createAndPlaceIOB(escape_name(pad_nc.name), pin_dir, pkg_pin, io_std)
        nc.rwCell.setSiteFixed(True)
    else:
        orig_type = nc.attrs["X_ORIG_TYPE"]
        uni_type = Unisim.valueOf(orig_type)
        # Expected NEXTPNR_BEL format: <SITE_TYPE>_<SITE_COORD>/<BEL>
        # Himbaechel format not yet supported!
        bel_loc = nc.attrs["NEXTPNR_BEL"]
        if bel_loc is not None:
            #print(f"Placing {uni_type} cell {nc.name} @ {bel_loc}")
            nc.rwCell = des.createAndPlaceCell(escape_name(nc.name), uni_type, bel_loc)
            nc.rwCell.setSiteFixed(True)
            nc.rwCell.setBELFixed(True)
        else:
            #print(f"{nc.name} does not have NEXTPNR_BEL")
            nc.rwCell = des.createCell(escape_name(nc.name), uni_type)

    assert nc.rwCell is not None, f"nc.rwCell is not assigned for {nc.name}"
    assert nc.rwCell.isPlaced()

    #print(f"{nc.rwCell.getName()} placed at {nc.rwCell.getSiteName()}")

    # process pin assignment
    # TODO: handle PS7/PS8
    if uni_type != Unisim.PS7 and uni_type != Unisim.PS8:
        # physical to logical pin map
        p2l_pin_map = nc.rwCell.getPinMappingsP2L()
        pins = p2l_pin_map.keys()
        # clear pin mappings for cell
        for pin in pins:
            nc.rwCell.removePinMapping(pin)
        for pin in nc.ports.values():
            if "X_ORIG_PORT_" + pin.name not in nc.attrs:
                continue
            orig_ports = nc.attrs["X_ORIG_PORT_" + pin.name].split(" ")
            for port in orig_ports:
                port = port.strip()
                if port:
                    #print(f"add pin mapping: P:{pin.name} <-> L:{port}")
                    nc.rwCell.addPinMapping(pin.name, port)

    # process cell parameters
    for param, param_value in nc.params.items():
        #print(f"{param}={param_value}")
        if param == "INIT":
            if uni_type == Unisim.LUT1:
                param_value = fixup_init(param_value, 1<<1)
            elif uni_type == Unisim.LUT2:
                param_value = fixup_init(param_value, 1<<2)
            elif uni_type == Unisim.LUT3:
                param_value = fixup_init(param_value, 1<<3)
            elif uni_type == Unisim.LUT4:
                param_value = fixup_init(param_value, 1<<4)
            elif uni_type == Unisim.LUT5 or uni_type == Unisim.RAMS32 or uni_type == Unisim.RAMD32:
                param_value = fixup_init(param_value, 1<<5)
            elif uni_type == Unisim.LUT6 or uni_type == Unisim.RAMS64E or uni_type == Unisim.RAMD64E:
                param_value = fixup_init(param_value, 1<<6)
            elif uni_type == Unisim.FDRE or uni_type == Unisim.FDSE or \
                    uni_type == Unisim.FDCE or uni_type == Unisim.FDPE:
                param_value = "1'b" + param_value[-1]
        elif uni_type == Unisim.RAMB18E2 or uni_type == Unisim.RAMB36E2:
            if param == "INIT_A" or param == "INIT_B" or param.startswith("SRVAL_"):
                param_value = fixup_init(param_value, 36 if uni_type == Unisim.RAMB36E2 else 18)
            elif param.startswith("INIT_") or param.startswith("INITP_"):
                param_value = fixup_init(param_value, 256)
        elif param.startswith("IS_") and param.endswith("_INVERTED"):
            param_value = fixup_init(param_value, 1).replace("h", "b")

        # TODO: check if WADR6,WADR7 ports are used
        if nc.rwCell.getType() == "RAMD64E" or nc.rwCell.getType() == "RAMS64E":
            nc.rwCell.addProperty("RAM_ADDRESS_MASK", "2'b11")
            nc.rwCell.addProperty("RAM_ADDRESS_SPACE", "2'b11")
        nc.rwCell.addProperty(param, param_value)

# create and connect each net
edif_top = des.getNetlist().getTopHierCellInst() # logical netlist
edif_gnd = EDIFTools.getStaticNet(NetType.GND, edif_top, des.getNetlist())
edif_vcc = EDIFTools.getStaticNet(NetType.VCC, edif_top, des.getNetlist())
for nn in ndes.nets.values():
    #print(f"Creating net {nn.name}")
    if nn.name == "$PACKER_VCC_NET":
        nn.rwNet = des.createNet(edif_vcc)
        des.addNet(nn.rwNet)
    elif nn.name == "$PACKER_GND_NET":
        nn.rwNet = des.createNet(edif_gnd)
        des.addNet(nn.rwNet)
    else:
        nn.rwNet = des.createNet(escape_name(nn.name))
        des.addNet(nn.rwNet)
    if nn.driver is not None and nn.driver.cell.rwCell is not None:
        if "X_ORIG_PORT_" + nn.driver.name not in nn.driver.cell.attrs:
            continue
        #print(f"connecting {nn.driver.cell.rwCell.getSiteName()}/{nn.driver.cell.rwCell.getBELName()}.{nn.driver.name} -> {nn.rwNet.getName()}")
        connect_phys_and_log(nn.rwNet, nn.driver.cell.rwCell, nn.driver.cell.attrs["X_ORIG_PORT_" + nn.driver.name])
    for user in nn.users:
        if user.cell.rwCell is not None:
            if "X_ORIG_PORT_" + user.name in user.cell.attrs:
                orig_ports = user.cell.attrs["X_ORIG_PORT_" + user.name].split(" ")
                for orig_port in orig_ports:
                    connect_phys_and_log(nn.rwNet, user.cell.rwCell, orig_port)
            else:
                # special case where no logical pin exists (e.g. A6 tied high for a fractured LUT)
                bel_pin = user.cell.rwCell.getBEL().getPin(user.name)
                if bel_pin is None:
                    continue
                site_pin_name = bel_pin.getConnectedSitePinName()
                si = user.cell.rwCell.getSiteInst()
                assert si is not None
                if site_pin_name is not None and si is not None:
                    if site_pin_name not in si.getSitePinNames():
                        nn.rwNet.createPin(site_pin_name, si)

# process net routing
for nn in ndes.nets.values():
    routing = nn.attrs["ROUTING"].split(";")
    print(f"Handling routing for {nn.name}")
    for i in range(0,len(routing)-2,3):
        dst_wire = routing[i] # DST
        pip_route = routing[i+1] # SRC->DST
        src_wire = pip_route.split("->")[0]
        print(f"\tSRC: {src_wire} -> DST: {dst_wire}")
        if src_wire.strip() == "" or dst_wire.strip() == "":
            #print("\t\tskipped")
            continue
        elif is_site_bel_pip(src_wire) and is_site_bel_pip(dst_wire):
            #print("\t\tintra site routing")
            # SRC: SITEWIRE/<SITE>_<SITE_COORD>/<BEL>_<PIN> 
            src_site_name,src_bel_name,src_bel_pin_name = parse_site_bel_pip(src_wire)
            # DST: SITEWIRE/<SITE>_<SITE_COORD>/<BEL>_<PIN>
            dst_site_name,dst_bel_name,dst_bel_pin_name = parse_site_bel_pip(dst_wire)
            assert src_site_name == dst_site_name, f"SRC site {src_site_name} != DST site {dst_site_name}"
            si = des.getSiteInstFromSiteName(src_site_name)
            assert si is not None,f"\t\tSiteInst {src_site_name} does not exist"
            si.routeIntraSiteNet(nn.rwNet, si.getBELPin(src_bel_name, src_bel_pin_name), si.getBELPin(dst_bel_name, dst_bel_pin_name))
        elif is_site_bel_pip(src_wire) and is_site_pin_pip(dst_wire):
            #print("\t\tintra site routing 2")
            # SRC: SITEWIRE/<SITE>_<SITE_COORD>/<BEL>_<PIN>
            src_site_name,src_bel_name,src_bel_pin_name = parse_site_bel_pip(src_wire)
            # DST: SITEWIRE/<SITE>_<SITE_COORD>/<SITE_PIN>
            dst_site_name,dst_site_pin_name = parse_site_pin_pip(dst_wire)
            assert src_site_name == dst_site_name, f"SRC site {src_site_name} != DST site {dst_site_name}"
            si = des.getSiteInstFromSiteName(src_site_name)
            assert si is not None,f"\t\tSiteInst {src_site_name} does not exist"
            si.routeIntraSiteNet(nn.rwNet, si.getBELPin(src_bel_name, src_bel_pin_name), si.getBELPin(dst_site_pin_name, dst_site_pin_name))
        elif is_site_pin_pip(src_wire) and is_site_bel_pip(dst_wire):
            #print("\t\tintra site routing 3")
            # SRC: SITEWIRE/<SITE>_<SITE_COORD>/<SITE_PIN>
            src_site_name,src_site_pin_name = parse_site_pin_pip(src_wire)
            # DST: SITEWIRE/<SITE>_<SITE_COORD>/<BEL>_<PIN>
            dst_site_name,dst_bel_name,dst_bel_pin_name = parse_site_bel_pip(dst_wire)
            assert src_site_name == dst_site_name, f"SRC site {src_site_name} != DST site {dst_site_name}"
            if src_site_name.startswith("IOB"):
                continue
            si = des.getSiteInstFromSiteName(src_site_name)
            assert si is not None,f"\t\tSiteInst {src_site_name} does not exist"
            si.routeIntraSiteNet(nn.rwNet, si.getBELPin(src_site_pin_name, src_site_pin_name), si.getBELPin(dst_bel_name, dst_bel_pin_name))
        elif is_site_pin_pip(dst_wire):
            #print("\t\tTODO: tile->site routing")
            # SRC: <TILE>_<TILE_COORD>/<TILE_WIRE>
            src_tile_name,src_wire_name = parse_tile_pip(src_wire)
            # DST: SITEWIRE/<SITE>_<SITE_COORD>/<SITE_PIN>
            dst_site_name,dst_site_pin_name = parse_site_pin_pip(dst_wire)
            src_tile = des.getDevice().getTile(src_tile_name)
            dst_si = des.getSiteInstFromSiteName(dst_site_name)
            if dst_si is None:
                dst_si = des.createSiteInst(des.getDevice().getSite(dst_site_name))
            dst_tile = des.getSiteInstFromSiteName(dst_site_name).getTile()
            #print(f"\t\tSRC Tile: {src_tile}, DST Tile: {dst_tile}")
            continue
        elif is_site_pin_pip(src_wire):
            #print("\t\tTODO: site->tile routing")
            # SRC: SITEWIRE/<SITE>_<SITE_COORD>/<SITE_PIN>
            src_site_name,src_site_pin_name = parse_site_pin_pip(src_wire)
            # DST: <TILE>_<TILE_COORD>/<TILE_WIRE>
            dst_tile_name,dst_wire_name = parse_tile_pip(dst_wire)
            src_tile = des.getSiteInstFromSiteName(src_site_name).getTile()
            dst_tile = des.getDevice().getTile(dst_tile_name)
            #print(f"\t\tSRC Tile: {src_tile}, DST Tile: {dst_tile}")
            continue
        else:
            print("\t\tTODO: tile->tile routing")
            # SRC: <TILE>_<TILE_COORD>/<TILE_WIRE>
            src_tile_name,src_wire_name = parse_tile_pip(src_wire)
            # DST: <TILE>_<TILE_COORD>/<TILE_WIRE>
            dst_tile_name,dst_wire_name = parse_tile_pip(dst_wire)
            src_tile = des.getDevice().getTile(src_tile_name)
            dst_tile = des.getDevice().getTile(dst_tile_name)
            src_wire_idx = src_tile.getWireIndex(src_wire_name)
            dst_wire_idx = dst_tile.getWireIndex(dst_wire_name)
            print(f"\t\tSRC Tile: {src_tile}, DST Tile: {dst_tile}")
            print(f"\t\tSRC Wire: {src_wire_idx}, DST Wire: {dst_wire_idx}")

In [ ]:
# TODO: compare NextpnrDesign and RapidWright Design netlist

In [ ]:
des.writeCheckpoint(out_dcp)